# a05.Sentiment

Citation:

Go, A., Bhayani, R. and Huang, L., 2009. Twitter sentiment classification using distant supervision. CS224N Project Report, Stanford, 1(12).

https://blog.cambridgespark.com/50-free-machine-learning-datasets-sentiment-analysis-b9388f79c124


The data is formatted under six fields, including 
- polarity, 
- tweet ID, 
- date, 
- query
- username and 
- the text of the tweet.

1. tokenization
2. adist
3. ngram
4. suggestion

In [1]:
# textblob
# snownlp
# https://ithelp.ithome.com.tw/articles/10203398

# https://textblob.readthedocs.io/en/latest/quickstart.html#quickstart
# https://github.com/isnowfy/snownlp

In [2]:
# ! pip install -U textblob
# ! python -m textblob.download_corpora
# ! pip install -U snownlp

In [3]:
from textblob import TextBlob

In [4]:
import importlib
import util
importlib.reload(util)
from util import *
# from vglib import vglib2
# import importlib
# importlib.reload(vglib2)
# #from <file_module> import <Class/Function>
# from vglib.vglib2 import Vglib

In [5]:
text = "I am happy today. I feel sad today."

blob = TextBlob(text)
# 用sententces方法將text斷句：blob.sentences
# 印出第一句的情緒分數 會輸出polarity(情感極性)以及subjectivity(主觀性)
sentiment1 = blob.sentences[0].sentiment
sentiment1

Sentiment(polarity=0.8, subjectivity=1.0)

In [6]:
{
    "polarity": sentiment1.polarity,
    "subjectivity": sentiment1.subjectivity,
}

{'polarity': 0.8, 'subjectivity': 1.0}

In [7]:
from snownlp import SnowNLP

In [8]:
text = "我今天很快樂。我今天很憤怒。"
s = SnowNLP(text)
# 用sententces方法將text斷句：
for sentence in s.sentences:
    print(sentence)

我今天很快樂
我今天很憤怒


In [9]:
s1 = SnowNLP(s.sentences[0])
s1.sentiments # 第1句的情緒分數,越接近1表示越正面

0.9268071116367116

In [10]:
{
    "polarity": s1.sentiments,
    "subjectivity": "NA",
}

{'polarity': 0.9268071116367116, 'subjectivity': 'NA'}

In [11]:
def sentiment(text):
    if 'eng' == detect_lang(text):
        blob = TextBlob(text)
        # 用sententces方法將text斷句：blob.sentences
        # 印出第一句的情緒分數 會輸出polarity(情感極性)以及subjectivity(主觀性)
        sentiment = [{
            "polarity": x.sentiment.polarity,
            "subjectivity": x.sentiment.subjectivity,
            "raw": str(x),
            "range": [-1,1],
        } for x in blob.sentences]
        return sentiment
    else:
        s = SnowNLP(text)

        sentiment = [{
            "polarity": SnowNLP(x).sentiments,
            "subjectivity": "NA",
            "raw": str(x),
            "range": [0,1],
        } for x in s.sentences]
        return sentiment

In [12]:
sentiment("我今天很快樂。我今天很憤怒。")

[{'polarity': 0.9268071116367116,
  'subjectivity': 'NA',
  'raw': '我今天很快樂',
  'range': [0, 1]},
 {'polarity': 0.1702660762575916,
  'subjectivity': 'NA',
  'raw': '我今天很憤怒',
  'range': [0, 1]}]

In [13]:
sentiment("I am happy today. I feel sad today.")

[{'polarity': 0.8,
  'subjectivity': 1.0,
  'raw': 'I am happy today.',
  'range': [-1, 1]},
 {'polarity': -0.5,
  'subjectivity': 1.0,
  'raw': 'I feel sad today.',
  'range': [-1, 1]}]

In [14]:
# End

# Training

In [15]:
from textblob.classifiers import NaiveBayesClassifier

In [16]:
# https://textblob.readthedocs.io/en/dev/classifiers.html
filename = "./dataset/trainingandtestdata/training.1600000.processed.noemoticon.csv"
MIN = 0
MAX = MIN + 1600000 - 10
idx = 0

train = []

with open(filename, mode="r", encoding="utf8") as f:
    try:
        for row in f:
            arr = row.split(",")

            lbl = ""
            if arr[0] == '"0"':
                lbl = "neg"
            elif arr[0] == '"4"':
                lbl = "pos"
            else:
                continue
                
            train.append((arr[5], lbl,))
            
            if idx > MAX:
                break
            idx = idx + 1
    except Exception as e:
        print(f'{idx}\n{e}')
        pass

In [50]:
import random
train0 = train
random.shuffle(train0)
print(f'{len(train0)}')
# cl = NaiveBayesClassifier(train0[:2400])
import pickle
f = open('my_classifier.pickle', 'rb')
cl = pickle.load(f)
f.close()


1601428


In [52]:
cl.classify("This is an amazing library!")

'pos'

In [53]:
import pickle
# f = open('my_classifier.pickle', 'wb')
# pickle.dump(cl, f)
# f.close()

In [41]:
len(train)

1601428

In [39]:
# https://textblob.readthedocs.io/en/dev/classifiers.html
filename = "./dataset/trainingandtestdata/testdata.manual.2009.06.14.csv"
MIN = 0
MAX = MIN + 1600000 - 10
idx = 0

test = []

with open(filename, mode="r", encoding="utf8") as f:
    try:
        for row in f:
            arr = row.split(",")

            lbl = ""
            if arr[0] == '"0"':
                lbl = "neg"
            elif arr[0] == '"4"':
                lbl = "pos"
            else:
                continue
                
            test.append((arr[5], lbl,))
            
            if idx > MAX:
                break
            idx = idx + 1
    except Exception as e:
        print(f'{idx}\n{e}')
        pass

In [40]:
test

[('"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool',
  'pos'),
 ('"Reading my kindle2...  Love it... Lee childs is good read."\n', 'pos'),
 ('"Ok', 'pos'),
 ('"@kenburbary You\'ll love your Kindle2. I\'ve had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)"\n',
  'pos'),
 ('"@mikefish  Fair enough. But i have the Kindle2 and I think it\'s perfect  :)"\n',
  'pos'),
 ('"@richardebaker no. it is too big. I\'m quite happy with the Kindle2."\n',
  'pos'),
 ('"Fuck this economy. I hate aig and their non loan given asses."\n', 'neg'),
 ('"Jquery is my new best friend."\n', 'pos'),
 ('"Loves twitter"\n', 'pos'),
 ('"how can you not love Obama? he makes jokes about himself."\n', 'pos'),
 ('"@Karoli I firmly believe that Obama/Pelosi have ZERO desire to be civil.  It\'s a charade and a slogan',
  'neg'),
 ('"House Correspondents dinner was last night whoopi', 'pos'),
 ('"Watchin Espn..Jus seen this new Nike Commerical with a P

In [42]:
cl.accuracy(test)

0.7075208913649025

# Testing

In [22]:
# testdata.manual.2009.06.14.csv
# - 0 == negative
# - 2 == neutral
# - 4 == positive

In [43]:
import pandas as pd
filename = "./dataset/trainingandtestdata/training.1600000.processed.noemoticon.csv"
#filename = "./dataset/trainingandtestdata/testdata.manual.2009.06.14.csv"
MIN = 1400000 - 100
MAX = MIN + 10000
idx = 0

collection = {
    "True": [],
    "False": [],
}

with open(filename, mode="r", encoding="utf8") as f:
    try:
        for row in f:
            if idx >= MIN and idx < MAX:
                print(f'{idx}')

                arr = row.split(",")

#                 sen_arr = sentiment(arr[5])
#                 scr_arr = [x["polarity"] for x in sen_arr]
#                 scr_sum = 0
#                 if len(scr_arr) > 0:
#                     scr_sum = sum(scr_arr) / len(scr_arr)

                clas = cl.classify(arr[5])
                if clas == "pos":
                    scr_sum = 4
                else:
                    scr_sum = 0

                mytest = ""
                if scr_sum == 0:
                    mytest = "2"
                elif scr_sum < 0:
                    mytest = "0"
                else:
                    mytest = "4"

                matched = f'"{mytest}"' == arr[0]
                collection[str(matched)].append(1)
                tweet = arr[5].strip('\n\r\t ')
                print(f"{tweet}\nmy score: {scr_sum}\t {mytest} vs {arr[0]}\n{matched}\n")



            if idx > MAX:
                break
            idx = idx + 1
    except Exception as e:
        print(f'{idx}\n{e}')
        pass

1399900
"Just ordered PIZZA and it's here. Be right back. "
my score: 0	 2 vs "4"
False

1399901
"I miss you sis "
my score: 0	 2 vs "4"
False

1399902
"There's a God of the computers an He allowed me to reinstall ITunes for my iPod "
my score: 0	 2 vs "4"
False

1399903
"@DinoGoesRawr good morning sunshine  how are you today?"
my score: 4	 4 vs "4"
True

1399904
"@_fatdad Ditto to the oatmeal
my score: 4	 4 vs "4"
True

1399905
"Had a blast at Mary's last night. And playing Rock Band I never died on guitar or drums. It was a success! "
my score: 4	 4 vs "4"
True

1399906
"yiiissss @LBKID2k9 has got twitter....woohoo!!  Hiiiii yaaaa babes!! wat u going to Cavan for? xoxo"
my score: 4	 4 vs "4"
True

1399907
"Watching #Bedtime #Stories with my kids "
my score: 4	 4 vs "4"
True

1399908
"@loribehr its on the life today website. "
my score: 4	 4 vs "4"
True

1399909
"i am so inloove again "
my score: 0	 2 vs "4"
False

1399910
"watching gossip girl and 90210 "
my score: 4	 4 vs "4"
True



"Reading Stephen King's Salem's Lot... "
my score: 4	 4 vs "4"
True

1399991
"Heading out for a day of kayaking in the beautiful waters around Ottawa
my score: 0	 2 vs "4"
False

1399992
"meeeow  byebye paint mah roooom with allisooooon!"
my score: 4	 4 vs "4"
True

1399993
"would actually marry brendon of coach trip "
my score: 4	 4 vs "4"
True

1399994
"@ScruffyPanther I'll double dare you with sprinkles on...... But want proof "
my score: 0	 2 vs "4"
False

1399995
"@sizemore picture disk cut to shape? didn't know it existed. You must've been a proper metal kid like myself! studded belts
my score: 0	 2 vs "4"
False

1399996
"Lisa has left the building
my score: 4	 4 vs "4"
True

1399997
"@stonexbaz i bet a glass of baileys with ice! Yeah i'm ok today thank you  hope you are too! x"
my score: 4	 4 vs "4"
True

1399998
"Bout to get ready and go watch my nieces ballet recital  she's too cute"
my score: 0	 2 vs "4"
False

1399999
"@jamhess very cool!!! I gotta clean...UGH!!! But resting

"@JasonBradbury Sorry mate
my score: 4	 4 vs "4"
True

1400070
"@PhilipTmw  good morning philip  have a nice day (nose porq te escribo en ingles jjaja)"
my score: 4	 4 vs "4"
True

1400071
"@beyondelsewhere Check your DMs pls "
my score: 4	 4 vs "4"
True

1400072
"@RDBONES p.s. - thanks very much for the Proverbs today.  "
my score: 4	 4 vs "4"
True

1400073
"@niklaswick Auch nicht schlecht. MacBook - Even Your Cat will love it
my score: 4	 4 vs "4"
True

1400074
"Just ordered my cap and gown.  61 days."
my score: 4	 4 vs "4"
True

1400075
"is heading to Canterbury for the Summer Ball tonight "
my score: 4	 4 vs "4"
True

1400076
"Congrats on getting married today!! Hope you found someone with Disney passes for the honeymoon  @luishandshake"
my score: 4	 4 vs "4"
True

1400077
"@natneagle its au thnx to u  hehe. enjoy UP
my score: 4	 4 vs "4"
True

1400078
"back to bed  7am is too early for a saturday!"
my score: 4	 4 vs "4"
True

1400079
"I cant believe how cool Emily is :x;X:x I love

"I'm soooooo fricken glad that I already have my Finals tickets! And that their not $100 a piece!  yay!"
my score: 0	 2 vs "4"
False

1400152
"@dboixel haha :p No I'm listening to damian Marley Haha Bob marleys son I think  "
my score: 4	 4 vs "4"
True

1400153
"That wasn't too bad... "
my score: 0	 2 vs "4"
False

1400154
"Finally. I have added songs on my cellphone and iPod! Finally. My iPods charged again. Now
my score: 0	 2 vs "4"
False

1400155
"Seeing 24 City at IFC and the Tian. Square exhibit on Elizabeth. Let me know if you're joining "
my score: 4	 4 vs "4"
True

1400156
"@ILIKErIKE Hi Rike
my score: 4	 4 vs "4"
True

1400157
"@nidheeshvaasu tch tch  lol.. get them a bottle of wine at least..."
my score: 4	 4 vs "4"
True

1400158
"Mornin' twitters  Watching tv with nessa
my score: 4	 4 vs "4"
True

1400159
"Thanks For the #FF @dr_zu  btw! como te fue anoche con lo de tropiezo?"
my score: 4	 4 vs "4"
True

1400160
"Thank you to @tattyted831 @drvictor @hallrm @SonjaJovan @Tori_

"@cLaRkKeNt22 u make it sound so simple haha. But fine...I guess I will  what r u up to?"
my score: 4	 4 vs "4"
True

1400245
"I'm still convinced that nothing compares to the iPhone. "
my score: 0	 2 vs "4"
False

1400246
"@TaraN STILL haven't fully decided though I am leaning towards the bb...everybody is addicted to theirs  How can I NOT join in the fun!?!"
my score: 4	 4 vs "4"
True

1400247
"@lexilovez Hey baby love
my score: 4	 4 vs "4"
True

1400248
"yeah! I love FNAC's Tickets haha :p ohhh it's okay  ohh you have a band ?  thats funny  what is &quot;Soundchekin'&quot; in french? :$"
my score: 4	 4 vs "4"
True

1400249
"Going to see Up! with my family now  going to jaxx later for the friday night boys cd release party!"
my score: 4	 4 vs "4"
True

1400250
"@kelliemcmullen people can think what they want
my score: 0	 2 vs "4"
False

1400251
"@MollyStahl: two years down
my score: 4	 4 vs "4"
True

1400252
"@dobb21 uh huh. Question of the day: do feelings have feelings?! "
my score

...

"@TheNotoriousJEN It's a lil family BBQ on morningside and 122nd.. Your welcome to come "
my score: 4	 4 vs "4"
True

1409496
"@jaceypants very good girl! Be positive "
my score: 4	 4 vs "4"
True

1409497
"waiting for meg'z and courtney "
my score: 4	 4 vs "4"
True

1409498
"is hoping for a killer start to the week tomorrow "
my score: 0	 2 vs "4"
False

1409499
"I love the new FNB album. "
my score: 4	 4 vs "4"
True

1409500
"Laters Twitter world. I'm off. Blessed day "
my score: 4	 4 vs "4"
True

1409501
"@alexcashcash lolololol. Can't Hardly Wait was on todayyy   all I thought about was Party In Your Bedroom video ahaha"
my score: 4	 4 vs "4"
True

1409502
"Wow I love this weather! It's to tweet for! "
my score: 4	 4 vs "4"
True

1409503
"I was amazed to see so many vehicles on the beach!  Even my little hybrid went off roading "
my score: 0	 2 vs "4"
False

1409504
"Headed to mt. Tam again with mtn bike in tow. This time i'm also bringing my pov camera. "
my score: 4	 4 vs "4"
True

"@officialnjonas what did i do to your heart is so good  is this your only twitter page? "
my score: 4	 4 vs "4"
True

1409576
"Gorgeous day in Pismo! Heading to the pier! "
my score: 4	 4 vs "4"
True

1409577
"Ugh. Going to the sock store.fml. Ill be there tilk five visit me "
my score: 0	 2 vs "4"
False

1409578
"@juicystar007 ooohh!!! take some pictures!! or do a haul vid!! "
my score: 0	 2 vs "4"
False

1409579
"happy Caturday
my score: 4	 4 vs "4"
True

1409580
"@ddlovato I wish you a pleasant day with Selena and Miley "
my score: 4	 4 vs "4"
True

1409581
"kyaking with whitney for the day
my score: 4	 4 vs "4"
True

1409582
"@massawyrm Perhaps the release of this week's /Filmcast: After Dark
my score: 4	 4 vs "4"
True

1409583
"Eating frozen yogurt while watching a devil wears prada!!! "
my score: 4	 4 vs "4"
True

1409584
"is enjoying a lazy
my score: 4	 4 vs "4"
True

1409585
"thanks for waiting patiently  I had to run an errand"
my score: 4	 4 vs "4"
True

1409586
"*huffs
my s

"On the cruise "
my score: 4	 4 vs "4"
True

1409662
"@Shannonyelland Nope still waiting. Was just saying you got those. Should've tweeted that better "
my score: 0	 2 vs "4"
False

1409663
"http://twitpic.com/6r7wv - @bnhuband &amp; me at grad lunch  good ol' can can- in all it's glory haha "
my score: 4	 4 vs "4"
True

1409664
"countdown to 80s party: tee minus 6 hours. "
my score: 4	 4 vs "4"
True

1409665
"@thefrankensteph just second round of shots is all "
my score: 4	 4 vs "4"
True

1409666
"@LucyD300 Tonight..... "
my score: 4	 4 vs "4"
True

1409667
"I want my prince charming...with great hair "
my score: 4	 4 vs "4"
True

1409668
"@TwoPeasandPod we rented it in the 80s and watched it over and over and over.  Def a childhood fave.  "
my score: 0	 2 vs "4"
False

1409669
"hmmm where should me and tanya eat laterrrr .. decisions decisions "
my score: 4	 4 vs "4"
True

1409670
"hey tweeties! yea i just woke up
my score: 4	 4 vs "4"
True

1409671
"kayaking with whitney for the day

"@YvetteObeney yep I know what u mean...maybe a food mood will strike  have a lovely evening !"
my score: 4	 4 vs "4"
True

1409751
"@colleencoover Do you need a lackey for the trip? My lackey skills are surpassed only by my ability to order train tickets in Italian! "
my score: 0	 2 vs "4"
False

1409752
"@ilovegeorgina It looks dark and a little gothic but still great. "
my score: 0	 2 vs "4"
False

1409753
"@BMolko Have a mad-dash amazing weekend! "
my score: 4	 4 vs "4"
True

1409754
"@ddlovato you have awesome songs on your new album! "
my score: 4	 4 vs "4"
True

1409755
"YAY! Will watch Pleasentville tonight
my score: 4	 4 vs "4"
True

1409756
"i slept
my score: 4	 4 vs "4"
True

1409757
"@ditzynicky k  and thanks!"
my score: 4	 4 vs "4"
True

1409758
"my pretty new dress  http://tinyurl.com/pot842"
my score: 4	 4 vs "4"
True

1409759
"That IS lame Liz lol
my score: 4	 4 vs "4"
True

1409760
"@_stevewoods Good Day to EVERYONE to you too!  U sound chipper 2 day!  love ur enthusia

"@inckognito Indeed. Thank you. I wish my dad was home so he could make me a loverly dinner.  x I can cook. I choose not too! I hate it."
my score: 0	 2 vs "4"
False

1409842
"There's a one-second lag in Star Cricket's coverage. Watch Doordarshan.  #WT20"
my score: 4	 4 vs "4"
True

1409843
"@spiddish Thank you "
my score: 4	 4 vs "4"
True

1409844
"@b_club Got my Bclub while I was in ATL for the FS tour! I'll take a pic of me rockin it out later today!  Have an amazing day!!!"
my score: 0	 2 vs "4"
False

1409845
"Help @QueenieCyrus get 130 followers or more by today "
my score: 4	 4 vs "4"
True

1409846
"@jason_mesnick hey jason can i get a shoutout
my score: 4	 4 vs "4"
True

1409847
"@literati_rain66 haha oh nifty well i'm glad i could help   so did jess tell you i got all that time off???!"
my score: 4	 4 vs "4"
True

1409848
"@KourtneyKardash  hello handsome lady lol"
my score: 4	 4 vs "4"
True

1409849
"@fearnecotton hi could you please play love shack by b52 thanks! "
my score:

In [44]:
good = sum(collection["True"])
good

7038

In [45]:
bad = sum(collection["False"])
bad

2962

In [46]:
good / (good + bad) * 100

70.38

In [47]:
bad / (good + bad) * 100

29.62